In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras import optimizers
from keras.layers import Dense, Dropout, LSTM, RepeatVector, TimeDistributed, Flatten
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

%matplotlib inline
sns.set(style="whitegrid")
warnings.filterwarnings("ignore")

# Set seeds to make the experiment more reproducible.
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(0)
seed(0)

Using TensorFlow backend.


### Load data

In [2]:
train = pd.read_csv('../input/X_train.csv')
labels = pd.read_csv('../input/y_train.csv')
test = pd.read_csv('../input/X_test.csv').sort_values(['series_id', 'measurement_number'])

### Join train features with labels

In [3]:
train = pd.merge(train, labels, how='inner').sort_values(['series_id', 'measurement_number'])
print(train.shape)
display(train.head())
display(train.describe().T)

(487680, 15)


row_id  series_id      ...        group_id        surface
0    0_0          0      ...              13  fine_concrete
1    0_1          0      ...              13  fine_concrete
2    0_2          0      ...              13  fine_concrete
3    0_3          0      ...              13  fine_concrete
4    0_4          0      ...              13  fine_concrete

[5 rows x 15 columns]

count     ...             max
series_id              487680.0     ...      3809.00000
measurement_number     487680.0     ...       127.00000
orientation_X          487680.0     ...         0.98910
orientation_Y          487680.0     ...         0.98898
orientation_Z          487680.0     ...         0.15571
orientation_W          487680.0     ...         0.15477
angular_velocity_X     487680.0     ...         2.28220
angular_velocity_Y     487680.0     ...         1.07910
angular_velocity_Z     487680.0     ...         1.38730
linear_acceleration_X  487680.0     ...        36.79700
linear_acceleration_Y  487680.0     ...        73.00800
linear_acceleration_Z  487680.0     ...        65.83900
group_id               487680.0     ...        72.00000

[13 rows x 8 columns]

### Feature engineering

In [4]:
def feat_eng(df):
    df['totl_anglr_vel'] = (df['angular_velocity_X']**2 + df['angular_velocity_Y']**2 + df['angular_velocity_Z'])** 0.5
    df['totl_linr_acc'] = (df['linear_acceleration_X']**2 + df['linear_acceleration_Y']**2 + df['linear_acceleration_Z'])**0.5
    df['totl_xyz'] = (df['orientation_X']**2 + df['orientation_Y']**2 + df['orientation_Z'])**0.5
    df['acc_vs_vel'] = df['totl_linr_acc'] / df['totl_anglr_vel']

feat_eng(train)
feat_eng(test)

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [5]:
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z


def perform_feature_engineering(df):
    x, y, z, w = df['orientation_X'].tolist(), df['orientation_Y'].tolist(), df['orientation_Z'].tolist(), df['orientation_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    df['euler_x'] = nx
    df['euler_y'] = ny
    df['euler_z'] = nz
    
    df['total_angle'] = np.sqrt(np.square(df['euler_x']) + np.square(df['euler_y']) + np.square(df['euler_z']))
    df['angle_vs_acc'] = df['total_angle'] / df['totl_linr_acc']
    df['angle_vs_vel'] = df['total_angle'] / df['totl_anglr_vel']
    
perform_feature_engineering(train)
perform_feature_engineering(test)

In [6]:
def mean_change_of_abs_change(x):
        return np.mean(np.diff(np.abs(np.diff(x))))

def mean_abs_change(x):
    return np.mean(np.abs(np.diff(x)))

def summarize_cols(df):
    cols = df.columns
    for col in cols:
        if col in ['row_id', 'series_id', 'measurement_number', 'group_id', 'surface']:
            continue
        df[col + '_mean'] = df.groupby(['series_id'])[col].mean()
        df[col + '_min'] = df.groupby(['series_id'])[col].min()
        df[col + '_max'] = df.groupby(['series_id'])[col].max()
        df[col + '_std'] = df.groupby(['series_id'])[col].std()
#         df[col + '_mad'] = df.groupby(['series_id'])[col].mad()
#         df[col + '_med'] = df.groupby(['series_id'])[col].median()
#         df[col + '_skew'] = df.groupby(['series_id'])[col].skew()
        df[col + '_max_to_min'] = df[col + '_max'] / df[col + '_min']
#         df[col + '_mean_abs_change'] = df.groupby('series_id')[col].apply(mean_abs_change)
#         df[col + '_mean_change_of_abs_change'] = df.groupby('series_id')[col].apply(mean_change_of_abs_change)
#         df[col + '_abs_max'] = df.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
#         df[col + '_abs_min'] = df.groupby('series_id')[col].apply(lambda x: np.min(np.abs(x)))
#         df[col + '_abs_mean'] = df.groupby('series_id')[col].apply(lambda x: np.mean(np.abs(x)))
#         df[col + '_abs_std'] = df.groupby('series_id')[col].apply(lambda x: np.std(np.abs(x)))

old_cols = list(train.columns)
summarize_cols(train)
summarize_cols(test)
add_cols = list(set(list(train.columns)) - set(old_cols))

In [7]:
train.fillna(0, inplace = True)
train.replace(-np.inf, 0, inplace = True)
train.replace(np.inf, 0, inplace = True)
test.fillna(0, inplace = True)
test.replace(-np.inf, 0, inplace = True)
test.replace(np.inf, 0, inplace = True)

### Turn data into series

In [8]:
def series_to_supervised(data, features, window=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data[features].shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in features]
    # Current timestep (t=0)
    cols.append(data)
    names += [(col) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    return agg

In [9]:
window = 3
features = ['orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W', 
            'angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z', 
            'linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z', 
            'totl_anglr_vel', 'totl_linr_acc', 'totl_xyz', 'acc_vs_vel' ,
            'series_id'] + add_cols
use_cols = features + ['surface']
# Train
train_series = series_to_supervised(train[use_cols], features, window=window)
train_series.dropna(inplace=True)        
train_series = train_series[(train_series['series_id'] == train_series['series_id(t-%s)' % window])]
display(train_series.head())

# Test
test_series = series_to_supervised(test[features], features, window=window)
test_series.fillna(test_series.mean(), inplace=True)

# Backwards inputation
for index, row in test_series.iterrows():
    for i in range(1, window+1):
        if row['series_id'] != row['series_id(t-%s)' % i]:
            for feature in features:
                if feature != 'series_id':
                    test_series.at[index, '%s(t-%s)' % (feature, i)] = row[feature]
display(test_series.head())

orientation_X(t-3)      ...              surface
3            -0.75853      ...        fine_concrete
4            -0.75853      ...        fine_concrete
5            -0.75853      ...        fine_concrete
6            -0.75852      ...        fine_concrete
7            -0.75852      ...        fine_concrete

[5 rows x 461 columns]

orientation_X(t-3)            ...              totl_anglr_vel_max_to_min
0           -0.025773            ...                                    0.0
1           -0.025683            ...                                    0.0
2           -0.025617            ...                                    0.0
3           -0.025773            ...                                    0.0
4           -0.025683            ...                                    0.0

[5 rows x 460 columns]

### Pre process the labels

In [10]:
n_labels =  train_series['surface'].nunique()
labels_names = train_series['surface'].unique()
le = LabelEncoder()
train_series['surface'] = le.fit_transform(train_series['surface'].values)

### Train/validation split

In [11]:
train_splited = []
val_splited = []
gp_surface = train.groupby(['surface', 'series_id'], as_index=False).count()[['surface', 'series_id']]
for surface in gp_surface['surface'].unique():
    df = gp_surface[gp_surface['surface'] == surface]
    tra, val = train_test_split(df, test_size=0.2, random_state=0)
    train_splited += list(tra['series_id'].values)
    val_splited += list(val['series_id'].values)
    
train_splited = pd.DataFrame(train_splited, columns=['series_id'])
val_splited = pd.DataFrame(val_splited, columns=['series_id'])

In [12]:
X_train = train_splited.merge(train_series)
X_val = val_splited.merge(train_series)

Y_train = X_train['surface']
Y_train = to_categorical(Y_train)
X_train.drop('surface', axis=1, inplace=True)
X_train = X_train

Y_val = X_val['surface']
Y_val = to_categorical(Y_val)
X_val.drop('surface', axis=1, inplace=True)
X_val = X_val

series_cols = list(X_train.filter(regex='series_id'))
X_train.drop(series_cols, axis=1, inplace=True)
X_val.drop(series_cols, axis=1, inplace=True)
test_series.drop(series_cols, axis=1, inplace=True)

print('Train shape', X_train.shape)
print('Validation shape', X_val.shape)
display(X_train.head())

Train shape (380500, 456)
Validation shape (95750, 456)


orientation_X(t-3)            ...              totl_anglr_vel_max_to_min
0            -0.98321            ...                                    0.0
1            -0.98321            ...                                    0.0
2            -0.98321            ...                                    0.0
3            -0.98321            ...                                    0.0
4            -0.98321            ...                                    0.0

[5 rows x 456 columns]

### Feature engineering

In [13]:
features = list(X_train.columns)
print('Train shape', X_train.shape)
print('Validation shape', X_val.shape)
display(X_train.describe().T)

Train shape (380500, 456)
Validation shape (95750, 456)


count     ...               max
orientation_X(t-3)                  380500.0     ...          0.989100
orientation_Y(t-3)                  380500.0     ...          0.988980
orientation_Z(t-3)                  380500.0     ...          0.155710
orientation_W(t-3)                  380500.0     ...          0.154770
angular_velocity_X(t-3)             380500.0     ...          2.282200
angular_velocity_Y(t-3)             380500.0     ...          1.079100
angular_velocity_Z(t-3)             380500.0     ...          1.387300
linear_acceleration_X(t-3)          380500.0     ...         36.797000
linear_acceleration_Y(t-3)          380500.0     ...         55.948000
linear_acceleration_Z(t-3)          380500.0     ...         65.839000
totl_anglr_vel(t-3)                 380500.0     ...          2.354958
totl_linr_acc(t-3)                  380500.0     ...         86.014069
totl_xyz(t-3)                       380500.0     ...          1.063410
acc_vs_vel(t-3)                     380500.0     ...       2644.116880
euler_x_mean(t-3)                   380500.0     ...          2.914980
totl_xyz_max(t-3)                   380500.0     ...          1.063220
euler_y_min(t-3)                    380500.0     ...          0.002634
orientation_X_mean(t-3)             380500.0     ...          0.988643
totl_linr_acc_max(t-3)              380500.0     ...        122.906924
euler_z_max(t-3)                    380500.0     ...          3.141576
euler_y_std(t-3)                    380500.0     ...          0.009889
angle_vs_acc_max(t-3)               380500.0     ...          0.000000
angular_velocity_X_mean(t-3)        380500.0     ...          0.193165
total_angle_std(t-3)                380500.0     ...          0.063030
angular_velocity_Y_max(t-3)         380500.0     ...          1.079100
linear_acceleration_Z_std(t-3)      380500.0     ...         19.953281
orientation_X_min(t-3)              380500.0     ...          0.988340
orientation_Z_mean(t-3)             380500.0     ...          0.152586
angular_velocity_Z_max_to_min(t-3)  380500.0     ...       1570.173985
totl_linr_acc_std(t-3)              380500.0     ...         20.478917
...                                      ...     ...               ...
orientation_W_max                   380500.0     ...          0.154770
linear_acceleration_Y_max_to_min    380500.0     ...       2003.296445
orientation_Y_max                   380500.0     ...          0.988980
orientation_Z_std                   380500.0     ...          0.007080
angular_velocity_Y_mean             380500.0     ...          0.341377
angular_velocity_Z_max              380500.0     ...          1.387300
angle_vs_acc_std                    380500.0     ...          0.000000
orientation_X_max_to_min            380500.0     ...        168.541728
linear_acceleration_Z_mean          380500.0     ...          0.000000
angle_vs_acc_min                    380500.0     ...         61.521600
orientation_Y_min                   380500.0     ...          0.988440
euler_z_max_to_min                  380500.0     ...          8.781905
angle_vs_vel_std                    380500.0     ...         26.013831
linear_acceleration_X_min           380500.0     ...          0.125160
orientation_Y_mean                  380500.0     ...          0.988642
angular_velocity_X_max_to_min       380500.0     ...          0.000000
euler_z_std                         380500.0     ...          3.140594
angular_velocity_Y_std              380500.0     ...          0.280561
linear_acceleration_Y_std           380500.0     ...         23.201073
linear_acceleration_X_max_to_min    380500.0     ...        229.722545
totl_linr_acc_max_to_min            380500.0     ...          0.000000
angle_vs_vel_mean                   380500.0     ...         29.048490
linear_acceleration_Z_min           380500.0     ...          0.000000
orientation_Z_max_to_min            380500.0     ...         24.168219
angular_velocity_X_std              380500.0     ...          0

### Scale the data

In [14]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(test_series)

In [15]:
subsequences = 2
timesteps = X_train.shape[1]//subsequences
X_train_series = X_train.reshape(X_train.shape[0], subsequences, (window+1), ((X_train.shape[1] // (window+1))//subsequences))
X_val_series = X_val.reshape(X_val.shape[0], subsequences, (window+1), ((X_val.shape[1] // (window+1))//subsequences))
X_test_series = X_test.reshape(X_test.shape[0], subsequences, (window+1), ((X_test.shape[1] // (window+1))//subsequences))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_val_series.shape)
print('Test set shape', X_test_series.shape)

Train set shape (380500, 2, 4, 57)
Validation set shape (95750, 2, 4, 57)
Test set shape (488448, 2, 4, 57)


### Model

In [16]:
epochs = 50
batch = 128
lr = 0.0001
adam = optimizers.Adam(lr)

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_series.shape[2], X_train_series.shape[3])))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(40, activation='relu'))
model.add(Dense(n_labels, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer=adam)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, None, 4, 64)       3712      
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 2, 64)       0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 40)                27040     
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 369       
Total params: 31,121
Trainable params: 31,121
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = model.fit(X_train_series, Y_train, validation_data=(X_val_series, Y_val), epochs=epochs, verbose=2)

Instructions for updating:
Use tf.cast instead.
Train on 380500 samples, validate on 95750 samples
Epoch 1/50
 - 91s - loss: 1.7650 - val_loss: 1.6860
Epoch 2/50
 - 88s - loss: 1.5265 - val_loss: 1.4579
Epoch 3/50
 - 87s - loss: 1.3760 - val_loss: 1.3676
Epoch 4/50
 - 87s - loss: 1.2842 - val_loss: 1.3277
Epoch 5/50
 - 87s - loss: 1.2146 - val_loss: 1.2324
Epoch 6/50
 - 89s - loss: 1.1606 - val_loss: 1.1834
Epoch 7/50
 - 87s - loss: 1.1155 - val_loss: 1.1618
Epoch 8/50
 - 87s - loss: 1.0763 - val_loss: 1.1364
Epoch 9/50
 - 87s - loss: 1.0409 - val_loss: 1.0664
Epoch 10/50
 - 88s - loss: 1.0085 - val_loss: 1.0388
Epoch 11/50
 - 87s - loss: 0.9812 - val_loss: 1.0329
Epoch 12/50
 - 87s - loss: 0.9546 - val_loss: 1.0148
Epoch 13/50
 - 88s - loss: 0.9313 - val_loss: 0.9918
Epoch 14/50
 - 87s - loss: 0.9099 - val_loss: 0.9755
Epoch 15/50
 - 87s - loss: 0.8893 - val_loss: 0.9659
Epoch 16/50
 - 87s - loss: 0.8717 - val_loss: 0.9671
Epoch 17/50
 - 88s - loss: 0.8533 - val_loss: 0.9377
Epoch 18/

#### Model loss plot

In [19]:
history_pd = pd.DataFrame.from_dict(history.history)
plt.figure(figsize=(20, 8))
ax = plt.axes()
ax.set_title('Model Loss')
sns.lineplot(data=history_pd)
plt.show()

#### Model evaluation

In [20]:
train_pred = model.predict_classes(X_train_series)
val_pred = model.predict_classes(X_val_series)
Y_train_max = np.argmax(Y_train, axis=1)
Y_val_max = np.argmax(Y_val, axis=1)

print('Train accuracy', accuracy_score(Y_train_max, train_pred))
print('Validation accuracy', accuracy_score(Y_val_max, val_pred))

Train accuracy 0.7915611038107753
Validation accuracy 0.7328355091383812


#### Model confusion matrix

In [21]:
cnf_matrix = confusion_matrix(Y_train_max, train_pred)
cnf_matrix_norm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
df_cm = pd.DataFrame(cnf_matrix_norm, index=labels_names, columns=labels_names)

plt.figure(figsize=(20, 7))
ax = plt.axes()
ax.set_title('Train')
sns.heatmap(df_cm, annot=True, fmt='.2f', cmap="Blues", ax=ax)
plt.show()

cnf_matrix = confusion_matrix(Y_val_max, val_pred)
cnf_matrix_norm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
df_cm = pd.DataFrame(cnf_matrix_norm, index=labels_names, columns=labels_names)

plt.figure(figsize=(20, 7))
ax = plt.axes()
ax.set_title('Validation')
sns.heatmap(df_cm, annot=True, fmt='.2f', cmap="Blues", ax=ax)
plt.show()

### Test predictions

In [22]:
predictions = model.predict_classes(X_test_series)
test['surface'] = le.inverse_transform(predictions)
df = test[['series_id', 'surface']]
df = df.groupby('series_id', as_index=False).agg(lambda x:x.value_counts().index[0])
df.to_csv('submission.csv', index=False)
df.head(10)

series_id                 surface
0          0  hard_tiles_large_space
1          1                  carpet
2          2                  carpet
3          3                  carpet
4          4                  carpet
5          5                concrete
6          6                  carpet
7          7              soft_tiles
8          8              soft_tiles
9          9                  carpet